In [1]:
import logging
import os
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
from tqdm.auto import tqdm
tqdm.pandas()  
from rouge import Rouge
rouge = Rouge()
from sklearn.model_selection import train_test_split
import re
from langdetect import detect
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
#ss

c:\Users\Azooo\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Azooo\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


c:\Users\Azooo\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Function to clean text by removing special characters and extra spaces
def clean_text(text):
    # Handle empty or NaN values
    if pd.isna(text):
        return ""
    
    # Convert to string and clean
    text = str(text)
    # Keep only Arabic characters and spaces
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
    # Remove extra spaces by splitting and joining with single space
    text = ' '.join(text.split())
    return text.strip()

# Function to process long texts by keeping important sentences
# This helps reduce the input size while maintaining key information
def process_text_length(text, max_length=2000):
    # If text is too long, keep only important parts
    if len(text) > max_length:
        # Split text into sentences using period as delimiter
        sentences = text.split('.')
        # Keep first sentence (main topic)
        processed_text = sentences[0] + '.'
        # Keep middle sentence - often contains supporting details
        mid_idx = len(sentences) // 2
        processed_text += sentences[mid_idx] + '.'
        # Keep last sentence (conclusion) - usually summarizes the main points
        processed_text += sentences[-1] + '.'
        return processed_text.strip()
    return text

# Function to check if text is in Arabic
# Uses langdetect library to identify the language
def is_arabic(text):
    try:
        return detect(text) == 'ar'  # 'ar' is the language code for Arabic
    except:
        return False  # Return False if language detection fails

# Main data loading and preprocessing
logger.info("Loading and preprocessing dataset...")
try:
    # Load the dataset
    df = pd.read_excel("Text summarization dataset.xlsx")
    
    # Fix column names and remove header
    df = df.iloc[1:].reset_index(drop=True)
    df.columns = ['summary', 'text']  # Rename columns for clarity
    logger.info("Successfully loaded and renamed columns")
    
    # Clean the data
    logger.info("Cleaning and processing texts...")
    df['text'] = df['text'].progress_apply(clean_text)
    df['summary'] = df['summary'].progress_apply(clean_text)
    df['text'] = df['text'].progress_apply(process_text_length)
    
    # Remove bad data
    df = df.dropna()  # Remove missing values
    df = df.drop_duplicates()  # Remove duplicates
    df = df[df['text'].str.len() >= 50]  # Remove very short texts
    
    # Calculate text statistics
    df['text_length'] = df['text'].str.len()
    df['summary_length'] = df['summary'].str.len()
    df['summary_ratio'] = df['summary_length'] / df['text_length']
    
    # Keep only reasonable summary lengths
    df = df[df['summary_ratio'].between(0.05, 0.3)]
    
    # Remove non-Arabic texts
    logger.info("Checking for non-Arabic texts...")
    df = df[df['text'].apply(is_arabic)]
    
    # Print dataset statistics
    logger.info(f"Final dataset size after cleaning: {len(df)} rows")
    logger.info("\nText length distribution after processing:")
    logger.info(df['text_length'].describe())
    logger.info("\nSummary ratio distribution:")
    logger.info(df['summary_ratio'].describe())
    
    # Split data for training
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
    
    # Convert pandas DataFrames to HuggingFace Dataset format
    # This format is required for the transformer model
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    
except FileNotFoundError:
    logger.error("Error: Dataset file 'Text summarization dataset.xlsx' not found.")
    logger.error("Please make sure the dataset file is in the same directory as the script.")
except Exception as e:
    logger.error(f"An error occurred: {str(e)}")
    logger.error("Please check the dataset format and try again.")

INFO:__main__:Loading and preprocessing dataset...
INFO:__main__:Successfully loaded and renamed columns
INFO:__main__:Cleaning and processing texts...


  0%|          | 0/29335 [00:00<?, ?it/s]

  0%|          | 0/29335 [00:00<?, ?it/s]

  0%|          | 0/29335 [00:00<?, ?it/s]

INFO:__main__:Checking for non-Arabic texts...
INFO:__main__:Final dataset size after cleaning: 15103 rows
INFO:__main__:
Text length distribution after processing:
INFO:__main__:count    15103.000000
mean      1344.492220
std       1045.321623
min         54.000000
25%        894.000000
50%       1246.000000
75%       1596.500000
max      18954.000000
Name: text_length, dtype: float64
INFO:__main__:
Summary ratio distribution:
INFO:__main__:count    15103.000000
mean         0.124586
std          0.058973
min          0.050000
25%          0.077683
50%          0.109297
75%          0.157426
max          0.300000
Name: summary_ratio, dtype: float64


### The compute_metrics, I never actually used. My hardware couldn't handel it. in the training arg. I seted the evalution_strategy to no. When you want to use it, do these changes: 
evaluation_strategy from "no" to "epoch", Enables evaluation at the end of each epoch

metric_for_best_model from "loss" to "rouge1", Uses ROUGE-1 score to determine the best model
 
 greater_is_better from False to True, Indicates that higher ROUGE scores are better 

 load_best_model_at_end from False to True, Saves the model checkpoint with the best ROUGE score 

In [3]:
# Load the AraBART model and tokenizer for Arabic text summarization
# Using moussaKam's pre-trained AraBART model which is specifically designed for Arabic text
model_name = "moussaKam/AraBART"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Check if GPU is available to speed up training
# This is important because training on GPU can be significantly faster
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to prepare our data for the model
# This function handles tokenization of both input text and target summaries
def preprocess_function(examples):
    # Convert input text to tokens
    inputs = tokenizer(
        examples["text"],
        max_length=512,  # Limit input length to 512 tokens
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Convert target summaries to tokens
    # Using max_length=128 for summaries as they should be shorter than input
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=128,  # Limit summary length to 128 tokens
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
    
    # Replace padding token id's of the labels by -100 so it's ignored by the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]
    
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

# Create datasets from our pandas DataFrames
# This converts our data into a format that HuggingFace can work with
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenize our datasets using the preprocessing function
# This applies our tokenization to all examples in the dataset
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

# Function to calculate ROUGE scores for evaluation
# ROUGE is a metric that measures the quality of summaries
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Convert predictions back to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Fix labels for decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Calculate ROUGE scores to measure summary quality
    # We use ROUGE  for comprehensive evaluation
    rouge_scores = rouge.get_scores(decoded_preds, decoded_labels, avg=True)
    
    return {
        'rouge1': rouge_scores['rouge-1']['f'],
        'rouge2': rouge_scores['rouge-2']['f'],
        'rougeL': rouge_scores['rouge-l']['f']
    }

# Set up training configuration
# These parameters are carefully chosen to balance training speed and model performance
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,  # Small batch size due to memory constraints
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    evaluation_strategy="no",  # Skip evaluation during training to save time
    save_strategy="epoch",
    metric_for_best_model="loss",
    greater_is_better=False,
    load_best_model_at_end=False,  # Don't load best model since we're not evaluating
    logging_dir='./logs',
    logging_first_step=True,
    disable_tqdm=False,
    fp16=True if torch.cuda.is_available() else False,  # Use mixed precision if GPU available
    gradient_accumulation_steps=16,  # Accumulate gradients to simulate larger batch size
    gradient_checkpointing=True,  # Save memory by recomputing activations
    optim="adamw_torch",
    max_grad_norm=1.0,
    dataloader_num_workers=0,
    dataloader_pin_memory=False
)

# Initialize the trainer with our model and data
# This sets up everything needed for training
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start the training process
# This will take some time depending on your hardware
logging.info("Starting training...")
trainer.train()

# Save the trained model for later use
# This allows us to use the model without retraining
trainer.save_model("./summarization_model")

c:\Users\Azooo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Azooo\anaconda3\Lib\site-packages\transformers\modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.ser

Map:   0%|          | 0/13592 [00:00<?, ? examples/s]

c:\Users\Azooo\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1511 [00:00<?, ? examples/s]

c:\Users\Azooo\anaconda3\Lib\site-packages\accelerate\accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
INFO:root:Starting training...


...

c:\Users\Azooo\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Azooo\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'loss': 3.3176, 'learning_rate': 5.89098532494759e-05, 'epoch': 2.35}


c:\Users\Azooo\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Azooo\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'loss': 2.7216, 'learning_rate': 6.49895178197065e-06, 'epoch': 4.71}


PermissionError: [WinError 5] Access is denied: './results\\tmp-checkpoint-1060' -> './results\\checkpoint-1060'